This example shows how to convert pretrained weights from Pytorch to the format that can be loaded by C++ based program, and used by the DLmodelzoo action set as well as dlpy `mzmodel()`. The function `convert_torchvision_model_weights` takes the parameter `model_name`, where we specify the full name of the model. Currently supported models are:
- resnet(resnet18, resnet34, resnet50, resnet101) 
- vgg(vgg11, vgg13, vgg16, vgg19)
- shufflenet(shufflenet_v2_x0_5, shufflenet_v2_x1_0)
- mobilenet(mobilenet_v2)

In [1]:
import swat as sw
from dlpy.mzmodel import *
from dlpy.utils import *
from dlpy.mzmodel_utils import *

In [2]:
s = sw.CAS('cloud.example.com', 5570)

In [3]:
s.table.addcaslib(activeonadd=False,
                  datasource={'srctype':'path'},
                  name='dnfs',
                  path='/example/path',
                  subdirectories=True)

NOTE: Cloud Analytic Services added the caslib 'dnfs'.


,Name,Type,Description,Path,Definition,Subdirs,Local,Active,Personal,Hidden,Transient
0,dnfs,PATH,,/example/path/,,1.0,1.0,0.0,0.0,0.0,0.0


In [4]:
s.table.loadTable(caslib='dnfs', path='data/cifar10_small.sashdat', 
    casout = dict(name = 'data', blocksize = '1', replace = True))

NOTE: Cloud Analytic Services made the file data/cifar10_small.sashdat available as table DATA in caslib CASUSER(userID).


[caslib]

 'CASUSER(userID)'

[tableName]

 'DATA'

[casTable]

 CASTable('DATA', caslib='CASUSER(userID)')

+ Elapsed: 0.0111s, user: 0.00686s, sys: 0.0027s, mem: 0.854mb

Use the SAS DLPy `convert_torchvision_model_weights()` function to convert Python model weights to weights that can be loaded by ModelZoo.

In [5]:
convert_torchvision_model_weights("resnet18", "path_to_local_directory/resnet18_cpp.pt")

NOTE: Pretrained weights loaded successfully.


C:\Users\userID\AppData\Local\Continuum\Miniconda3\lib\site-packages\torch\jit\_script.py:890: UserWarning: `optimize` is deprecated and has no effect. Use `with torch.jit.optimized_execution() instead
  warnings.warn(


NOTE: Pretrained weights saved successfully.


In [6]:
model = MZModel(conn=s, model_type = "torchNative", model_name="resnet", model_subtype="resnet18", 
                num_classes=10, model_path = "path_to_local_directory/resnet18_cpp.pt")

In [7]:
model.add_image_transformation(image_size='256 256')

In [8]:
optimizer=Optimizer(seed=54321, 
                    algorithm=SGDSolver(lr=0.001, momentum=0.9),
                    batch_size=128,
                    max_epochs=10                   
                    )

In [9]:
model.train(table="data", inputs="_image_", targets="labels", gpu=[0], optimizer=optimizer, index_variable='labels')

NOTE: Using this GPU List on your-server.unx.company.com: [0 ].
NOTE: dlx executable command : /r/ge.unx.sas.com/vol/vol610/u61/userID/tkext/com/laxnd/dlxexe -controlpid 44409 -session 25
NOTE: Worker rank 0: your-server.unx.company.com, using up to 1 GPU devices.
NOTE: Using your-server.unx.company.com: GPU 0 processes up to 128 records at a time.
NOTE: Starting dlxexe process on worker your-server.unx.company.com, rank/threadID [0/0], pid= 2917 device= GPU:0
NOTE: Learning Rate:   0.001 Momentum:     0.9 Weight Decay:       0 Dampening:       0 BatchSize: 128 Rank: 0 Device: 0 Loss: 138.673
NOTE: The action completed successfully.


,Epoch,Loss,MCE
0,0.0,1574.979976,0.927536
1,1.0,1181.424218,0.864227
2,2.0,902.265424,0.776081
3,3.0,708.099181,0.625714
4,4.0,553.206810,0.496875
5,5.0,415.701191,0.347079
6,6.0,300.095903,0.153551
7,7.0,226.271760,0.060484
8,8.0,177.045486,0.052632
9,9.0,138.673359,0.028689
